---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [9]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc, columns=['text']) 
df[120:200]

,text
120,12/22/98 CPT Code: 90801 - Psychiatric Diagnos...
121,10/02/96 Age:\n
122,11/05/90 CPT Code: 90792: With medical services\n
123,5/04/77 CPT Code: 90792: With medical services\n
124,2/27/96 Communication with referring physician...
125,s The patient is a 44 year old married Caucasi...
126,.10 Sep 2004 - Intake at EEC for IOP but did n...
127,see above and APS eval of 26 May 1982 Social H...
128,Tbooked for intake appointment at Sierra Vista...
129,06 May 1972 SOS-10 Total Score:\n


In [42]:
import numpy as np


def date_sorter():

    
    test_1 = pd.Series(['04/20/2009', '04/20/09', '4/20/09', '4/3/09',
                       '6/2008', '12/2009',
                      '2009', '2010',
                       ])
    
#     test_2 = pd.Series(['Mar-20-2009', 'Mar 20, 2009', 'March 20, 2009', 'Mar. 20, 2009', 'Mar 20 2009',
#                    'Feb 2009', 'Sep 2009', 'Oct 2010',
#                    'Mar 20th, 2009', 'Mar 21st, 2009', 'Mar 22nd, 2009'
#                   ])
    
#     test_3 = pd.Series(['20 Mar 2009', '20 March 2009', '20 Mar. 2009', '20 March, 2009'
#                     ])
   
    
    # Your code here
    extract_0 = df.text.str.extractall(r'(?:(?P<MONTH>\d{1,2})[/-])?(?:(?P<DAY>\d{1,2})[/-])?(?P<YEAR>[12]\d{3})')
    extract_1 = df.text.str.extractall(r'(?:(?P<MONTH>\d{1,2})[/-])(?:(?P<DAY>\d{1,2})[/-])(?P<YEAR>\d{2,4})')
    extract_2 = df.text.str.extractall(r'(?P<MONTH>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|(?:\.? ))(?:(?P<DAY>\d{1,2})(?:[a-z]{2})?(?:-|(?:,? )))?(?P<YEAR>[12]\d{3})')
    extract_3 = df.text.str.extractall(r'(?:(?P<DAY>\d{1,2})(?:[a-z]{2})?(?:-|(?:,? )))?(?P<MONTH>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|(?:[,\.]? ))(?P<YEAR>[12]\d{3})')


    MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
    digits = ("{:02}".format(month) for month in range(1, 13))
    MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
    
    result = extract_3.append(extract_2).append(extract_1).append(extract_0)
    

    result['YEAR']=result['YEAR'].apply(lambda x: '19'+x if len(x)<=2 else x)
    result['MONTH']=result['MONTH'].apply(lambda x: MONTH_TO_DIGITS[x] if x in MONTH_PREFIXES else x)
    result['MONTH'] = result['MONTH'].replace(np.nan, 1)
    result['DAY'] = result['DAY'].replace(np.nan, 1)
    

    result.index.names = ['index', 'match']
    result = result[['YEAR','MONTH','DAY']].astype(int)
    
    result = result.reset_index()
    result = result.drop_duplicates("index")
    
    result = result.sort_values(by=['YEAR','MONTH','DAY'])
    result = pd.Series(result["index"].values)
    return result# Your answer here

In [43]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64